In [1]:
import matplotlib.pyplot as plt
import numpy as np

import methods
import lab2.methods as methods2

ModuleNotFoundError: No module named 'lab1'

In [ ]:
from scipy.optimize import approx_fprime
import math

np.random.seed(12)


def generate_points(n, f, rg=1):
    X = rg * np.random.rand(n)
    y = []
    X_err = X + 0.2 * np.random.randn(n)
    for x in X_err:
        y.append(f(x))
    return X, np.asarray(y)


def mse(f, w, X, y):
    diff = 0
    for i in range(0, len(X)):
        diff += (f(w)(X[i]) - y[i]) ** 2
    return diff


def mse_func(f, X, y):
    return lambda w: mse(f, w, X, y)


def grad_calculator(x, func, dim):
    h = 1e-5
    res = []
    for i in range(dim):
        delta = np.zeros(dim)
        delta[i] = h
        res.append((func(x + delta) - func(x - delta)) / (2 * h))
    return np.asarray(res)

In [ ]:
def grad_func(f, dim):
    return lambda x: grad_calculator(x, f, dim)


def func(p):
    return lambda x: p[0] + p[1] * math.sin(3 * x) / x + p[2] * x


def target_func(x):
    return func([1, 1, 0.1])(x)


n = 10
dim = 3
rg = 10
(X, y) = generate_points(n, target_func, rg)
f = mse_func(func, X, y)
grad = grad_func(f, dim)
hess = lambda x0: approx_fprime(x0, lambda t: approx_fprime(t, f))

lr = lambda x: 1
start = np.zeros(dim)

In [ ]:
def make_plot(w1=None, name1=None, w2=None, name2=None, w3=None, name3=None, w4=None, name4=None, w5=None, name5=None):
    fig, ax = plt.subplots()
    plt.plot(X, y, 'b.')
    ls = []
    names = []
    t1 = np.arange(0.0, rg + 0.01, 0.1)
    if not w1 is None:
        Y = []
        for x in t1:
            Y.append(func(w1)(x))
        l1, = plt.plot(t1, Y, '-', color='r', linewidth=1, alpha=0.8)
        ls.append(l1)
        names.append(name1)
    if not w2 is None:
        Y = []
        for x in t1:
            Y.append(func(w2)(x))
        l2, = plt.plot(t1, Y, '-', color='y', linewidth=1, alpha=0.8)
        ls.append(l2)
        names.append(name2)
    if not w3 is None:
        Y = []
        for x in t1:
            Y.append(func(w3)(x))
        l3, = plt.plot(t1, Y, '-', color='g', linewidth=1, alpha=0.8)
        ls.append(l3)
        names.append(name3)
    if not w4 is None:
        Y = []
        for x in t1:
            Y.append(func(w4)(x))
        l4, = plt.plot(t1, Y, '-', color='orange', linewidth=1, alpha=0.8)
        ls.append(l4)
        names.append(name4)
    if not w5 is None:
        Y = []
        for x in t1:
            Y.append(func(w5)(x))
        l5, = plt.plot(t1, Y, '-', color='pink', linewidth=1, alpha=0.8)
        ls.append(l5)
        names.append(name5)
    ax.legend(ls, names, loc='upper right', shadow=True)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()

In [ ]:
def print_table(infos):
    for info in infos:
        print("\hline")
        print("{} & {} & {} \\".format(info[0][0], info[1], info[2]))

In [ ]:
infos = []

infos.append(methods.bfgs(f, grad, start))
w1 = infos[-1][0][-1]

infos.append(methods2.sgd_adam(f, grad, start))
w2 = infos[-1][0][-1]

infos.append(methods2.sgd_rmsprop(f, grad, start))
w3 = infos[-1][0][-1]

infos.append(methods.trust_region_dogleg(f, grad, hess, start))
w4 = infos[-1][0][-1]

def gauss_f(p):
    return [func(p)(x_i) for x_i in X]

def jacobian(p):
    n = len(X)
    m = len(p)
    h = 0.001
    J = np.zeros((n, m))

    for i in range(n):
        for j in range(m):
            p_minus = p.copy()
            p_plus = p.copy()
            p_minus[j] -= h
            p_plus[j] += h
            J[i, j] = (func(p_plus)(X[i]) - func(p_minus)(X[i])) / (2 * h)
    return J

infos.append(methods.gauss_newton(gauss_f, jacobian, X, y, start))
w5 = infos[-1][0][-1]

infos.append(methods.gauss_newton(func, grad_func(func, dim), X, y, start))
print_table(infos)
make_plot(w1, "bfgs", w2, "adam", w3, "rmsprop", w4, "dogleg")